In [ ]:
from pathlib import Path

path = Path("./data")


[PosixPath('data/dara_pose_r_cam_u_lincheng'), PosixPath('data/data_pose_r_cam_u_lincheng2'), PosixPath('data/data_pose_r_cam_u_3'), PosixPath('data/data_pose_r_cam_u_1'), PosixPath('data/data_ss_2'), PosixPath('data/data_pose_r_cam_u_lincheng'), PosixPath('data/data_ss_'), PosixPath('data/data_pose_r_cam_u_lin'), PosixPath('data/data_pose_r_cam_u_2'), PosixPath('data/dara_pose_r_cam_u_lin'), PosixPath('data/resources'), PosixPath('data/data_pose_r_cam_u_4'), PosixPath('data/data_pose_r_cam_u_lincheng3')]


In [ ]:
import json
from pathlib import Path
from collections import OrderedDict

file_contents = {}
for json_f in path.rglob("sa*.json"):
    json_contents = json.loads(json_f.read_text())
    print(len(json_contents))
    file_contents.update(json_contents)
    # break

# Sort the file_contents by the 'prompt' field
sorted_contents = OrderedDict(
    sorted(file_contents.items(), key=lambda item: int(item[1]['prompt']))
)

output = Path("./data") / "lincheng"
output.mkdir(exist_ok=True)
with open(output / "saved_data.json", "w") as f:
    json.dump(sorted_contents, f, indent=4)


# yolo finetun using different dataset

In [ ]:
from tqdm import tqdm

from ultralytics.utils.checks import check_requirements
from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywhn

import numpy as np
from pathlib import Path

check_requirements(('pycocotools>=2.0',))
from pycocotools.coco import COCO

# Make Directories
dir = Path('/home/capre/disk_4/yutao/ultralytics/datasets/Objects365')  # dataset root dir
for p in 'images', 'labels':
    (dir / p).mkdir(parents=True, exist_ok=True)
    for q in 'train', 'val':
        (dir / p / q).mkdir(parents=True, exist_ok=True)

# Train, Val Splits
for split, patches in [('train', 50 + 1), ('val', 43 + 1)]:
    print(f"Processing {split} in {patches} patches ...")
    images, labels = dir / 'images' / split, dir / 'labels' / split

    # Download
    url = f"https://dorc.ks3-cn-beijing.ksyun.com/data-set/2020Objects365%E6%95%B0%E6%8D%AE%E9%9B%86/{split}/"
    if split == 'train':
        download([f'{url}zhiyuan_objv2_{split}.tar.gz'], dir=dir)  # annotations json
        download([f'{url}patch{i}.tar.gz' for i in range(patches)], dir=images, curl=True, threads=8)
    elif split == 'val':
        download([f'{url}zhiyuan_objv2_{split}.json'], dir=dir)  # annotations json
        download([f'{url}images/v1/patch{i}.tar.gz' for i in range(15 + 1)], dir=images, curl=True, threads=8)
        download([f'{url}images/v2/patch{i}.tar.gz' for i in range(16, patches)], dir=images, curl=True, threads=8)

    # Move
    for f in tqdm(images.rglob('*.jpg'), desc=f'Moving {split} images'):
        f.rename(images / f.name)  # move to /images/{split}

    # Labels
    coco = COCO(dir / f'zhiyuan_objv2_{split}.json')
    names = [x["name"] for x in coco.loadCats(coco.getCatIds())]
    for cid, cat in enumerate(names):
        catIds = coco.getCatIds(catNms=[cat])
        imgIds = coco.getImgIds(catIds=catIds)
        for im in tqdm(coco.loadImgs(imgIds), desc=f'Class {cid + 1}/{len(names)} {cat}'):
            width, height = im["width"], im["height"]
            path = Path(im["file_name"])  # image filename
            try:
                with open(labels / path.with_suffix('.txt').name, 'a') as file:
                    annIds = coco.getAnnIds(imgIds=im["id"], catIds=catIds, iscrowd=None)
                    for a in coco.loadAnns(annIds):
                        x, y, w, h = a['bbox']  # bounding box in xywh (xy top-left corner)
                        xyxy = np.array([x, y, x + w, y + h])[None]  # pixels(1,4)
                        x, y, w, h = xyxy2xywhn(xyxy, w=width, h=height, clip=True)[0]  # normalized and clipped
                        file.write(f"{cid} {x:.5f} {y:.5f} {w:.5f} {h:.5f}\n")
            except Exception as e:
                print(e)


In [2]:
import os
from ultralytics import YOLO
# os.chdir("/home/capre/disk_4/yutao/ultralytics")
# Load a model
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")  # load a pretrained model (recommended for training)

# Train the model
# results = model.train(data="Objects365.yaml", epochs=100, imgsz=640)
image_path = '/home/capre/disk_4/yutao/data/resources/0a7ee4d0cf344e15a81c68be0be1fb96_color_2.png'
res= model.predict(source=image_path, save=False, verbose=False)
res[0].show()

# yolo finetune for breast seg

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/home/capre/disk_4/yutao/breast-seg/dataset3/dataset.yaml", epochs=100, imgsz=640)

# masks filename align

In [ ]:
import os

def add_leading_zeros_to_masks(masks_dir, total_digits=4):
    """
    将 masks_dir 中的文件名加上前导零，使其成为指定长度的数字字符串。

    参数：
    - masks_dir: 掩码文件夹的路径
    - total_digits: 文件名应达到的总位数，默认为4
    """
    for filename in os.listdir(masks_dir):
        # 分离文件名和扩展名
        basename, extension = os.path.splitext(filename)
        try:
            # 将文件名转换为整数，以确保文件名是数字
            number = int(basename)
            # 格式化新的文件名，添加前导零
            new_basename = f"{number:0{total_digits}d}"
            new_filename = new_basename + extension
            # 构建完整的源和目标路径
            src = os.path.join(masks_dir, filename)
            dst = os.path.join(masks_dir, new_filename)
            # 重命名文件
            os.rename(src, dst)
            print(f"重命名：{filename} -> {new_filename}")
        except ValueError:
            print(f"跳过非数字文件名：{filename}")

# 使用示例：
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'  # 替换为您的 masks 文件夹路径
add_leading_zeros_to_masks(masks_dir)


# yolo dataset make

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import shutil

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset2'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        height, width = mask.shape
        # Threshold mask to binary
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= width
                normalized_contour[:, 1] /= height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file (class index is 0)
                line = '0 ' + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

        # Visualization (optional)
        # if random.random() < 0.05:  # Adjust the probability as needed
        #     # Plot image and contours
        #     plt.figure(figsize=(10, 10))
        #     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #     for contour in contours:
        #         contour = contour.squeeze()
        #         plt.plot(contour[:, 0], contour[:, 1], linewidth=2)
        #     plt.title(f"{phase} - {image_file}")
        #     plt.axis('off')
        #     plt.show()

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    f.write("  0: object\n")


# add label from predict

In [ ]:
import os
import cv2
import numpy as np
import random
import shutil
from ultralytics import YOLO

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset3'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Load the YOLO model
model = YOLO('yolo11x-seg.pt')  # Replace with the correct path to your model

# Get class names and assign a new class ID for your mask label
class_names = model.names  # A dictionary {class_id: class_name}
max_class_id = max(class_names.keys())
mask_class_id = max_class_id + 1
class_names[mask_class_id] = 'breast'  # Replace 'breast' with your class name

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        original_height, original_width = image.shape[:2]

        # Run the model to get predictions
        results = model.predict(source=image, save=False, verbose=False)
        result = results[0]

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            # Process predicted masks
            if hasattr(result, 'masks') and result.masks is not None:
                pred_masks = result.masks.data.cpu().numpy()
                pred_classes = result.boxes.cls.cpu().numpy().astype(int)
                for mask_pred, class_id in zip(pred_masks, pred_classes):
                    # Convert mask to binary image
                    mask_pred = (mask_pred > 0.5).astype(np.uint8) * 255

                    # Resize mask back to original image size
                    mask_pred_resized = cv2.resize(mask_pred, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

                    # Find contours
                    contours, _ = cv2.findContours(mask_pred_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    for contour in contours:
                        # Simplify contour
                        epsilon = 0.001 * cv2.arcLength(contour, True)
                        contour = cv2.approxPolyDP(contour, epsilon, True)

                        # Flatten contour array
                        contour = contour.squeeze()
                        if contour.ndim != 2:
                            continue  # Skip if contour is not 2D

                        # Normalize coordinates
                        normalized_contour = contour.astype(np.float32)
                        normalized_contour[:, 0] /= original_width
                        normalized_contour[:, 1] /= original_height

                        # Flatten and convert to list
                        contour_list = normalized_contour.flatten().tolist()

                        # Write to file
                        line = f"{class_id} " + ' '.join(map(str, contour_list))
                        f.write(line + '\n')
            else:
                print(f"No predicted masks for image {image_file}")

            # Process your own mask
            # Threshold mask to binary
            _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # Find contours
            contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= original_width
                normalized_contour[:, 1] /= original_height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file with mask_class_id
                line = f"{mask_class_id} " + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    for class_id in sorted(class_names.keys()):
        f.write(f"  {class_id}: {class_names[class_id]}\n")


# LEO Dataset Processing Pipeline

# Original Data Process (for xyz rxyz is invert)

In [11]:
from pathlib import Path
import json
import numpy as np
from scipy.spatial.transform import Rotation as R
import pandas as pd
data_path = '/home/capre/disk_4/yutao/data'

def switch_xyzrxyz(rxyzxyz:list):
    r_xyz, xyz = rxyzxyz[0:3], rxyzxyz[3:6]
    xyzrxyz = np.hstack((xyz, r_xyz))
    return xyzrxyz.tolist()

# print(switch_xyzrxyz([1,2,3,4,5,6]))

def transform_pose(pose: list, T_cam_to_base):
    pose: np.ndarray = np.array(pose)
    t_xyz = pose[0:3]
    r_xyz = pose[3:6]
    rotation_matrix = R.from_euler('xyz', r_xyz.reshape(1, 3), degrees=False).as_matrix().reshape(3, 3)
    T_end_to_base = np.eye(4)
    T_end_to_base[:3, :3] = rotation_matrix
    T_end_to_base[:3, 3] = t_xyz.ravel()
    T_base_to_cam = np.linalg.inv(T_cam_to_base)

    T_cam_to_end = T_base_to_cam @ T_end_to_base
    new_t = T_cam_to_end[:3, 3]
    new_r = R.from_matrix(T_cam_to_end[:3, :3]).as_euler('xyz', degrees=False)
    xyzrxrzry = np.hstack((new_r, new_t.reshape(-1)))
    return xyzrxrzry.tolist()


path = Path(data_path)
resources_path: Path = path / 'resources'
all_data = {}
x_values, y_values, z_values = [], [], []

for json_f in path.rglob("*saved_data.json"):
    related_calib_json = json_f.parent / 'Calibration_results' / 'calibration_results.json'
    if related_calib_json.exists():
        calib = json.load(open(related_calib_json))
        saved_data = json.load(open(json_f))
        save = True
        for each_record in saved_data:
            for each_image in saved_data[each_record]['color_files']:
                if not resources_path.joinpath(each_image).exists():
                    save = False
                    break
            if not save:
                break
            
            for idx, pose in enumerate(saved_data[each_record]['poses']):
                saved_data[each_record]['poses'][idx]['pose0'] = switch_xyzrxyz(saved_data[each_record]['poses'][idx]['pose0'])
                if saved_data[each_record]['poses'][idx]['pose1'] is not None:
                    saved_data[each_record]['poses'][idx]['pose1'] = switch_xyzrxyz(saved_data[each_record]['poses'][idx]['pose1'])


            for idx, pose in enumerate(saved_data[each_record]['pose']):
                saved_data[each_record]['pose'][idx] = switch_xyzrxyz(pose)
                x_values.append(saved_data[each_record]['pose'][idx][0])
                y_values.append(saved_data[each_record]['pose'][idx][1])
                z_values.append(saved_data[each_record]['pose'][idx][2])
            # print(saved_data[each_record])
            all_data[each_record] = saved_data[each_record]
            # break

# Compute min and max for x, y, z
min_x, max_x = min(x_values), max(x_values)
min_y, max_y = min(y_values), max(y_values)
min_z, max_z = min(z_values), max(z_values)

# Print the results
print(f"Min x: {min_x}, Max x: {max_x}")
print(f"Min y: {min_y}, Max y: {max_y}")
print(f"Min z: {min_z}, Max z: {max_z}")


Min x: -0.41059924008326715, Max x: 0.03631917139252827
Min y: -0.7661798246708781, Max y: 0.29486756366552436
Min z: 0.01451047114314541, Max z: 0.8342742760627375


# map id to real prompts

In [12]:
prompts_map = pd.read_csv(path / 'task_list.csv')
prompts_map['ID'] = prompts_map['ID'].astype(int).astype(str)

for each_record in all_data:
    all_data[each_record]['prompt'] = prompts_map[prompts_map['ID'] == all_data[each_record]['prompt']]['Prompts'].values[0]

# select data

In [19]:
import random

half_data_len = len(all_data) // 2
random_keys = random.sample(list(all_data.keys()), half_data_len)
half_data = {key: all_data[key] for key in random_keys}
print(len(half_data))
json.dump(half_data, open(path / 'half_data.json', 'w'), indent=4)

half_half_data_len = len(all_data) // 2 // 2
random_keys = random.sample(list(all_data.keys()), half_half_data_len)
half_half_data = {key: all_data[key] for key in random_keys}
print(len(half_half_data))
json.dump(half_half_data, open(path / 'half_half_data.json', 'w'), indent=4)

170
85


In [ ]:
import cv2
cv2.goodFeaturesToTrack

In [14]:
import plyfile
data = plyfile.PlyData.read('/home/capre/disk_4/yutao/data/resources/0a7ee4d0cf344e15a81c68be0be1fb96_point_cloud_1.ply')

In [ ]:
# ! python launch.py task=tuning_vla note=tuning_vla pretrained_ckpt_path=/home/capre/disk_4/yutao/leo/ckpts clip_txt_guidance.flag=True

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar'), PlyProperty('segment_id', 'int')), count=208304, comments=[]),), text=True, byte_order='=', comments=[], obj_info=[])

# recalculate the segmentation

In [ ]:
import os
from ultralytics import YOLO
from plyfile import PlyElement, PlyData
import open3d as o3d
import open3d.core as o3c
# os.chdir("/home/capre/disk_4/yutao/ultralytics")
# Load a model
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")  # load a pretrained model (recommended for training)
o3d_device = o3d.core.Device("CUDA:0")
from utils.segmentation import segment_pcd_from_2d
intrinsic = np.array([
    [
      610.5961520662408,
      0.0,
      639.8919938587554
    ],
    [
      0.0,
      617.4130735412369,
      358.3889735843055
    ],
    [
      0.0,
      0.0,
      1.0
    ]
  ]).T

for idxx, each_record in enumerate(all_data):
    print(f"Processing {idxx}/{len(all_data)}: {each_record}")
    for idx, each_images in enumerate(all_data[each_record]['color_files']):
        # print(each_image)
        depth = resources_path / all_data[each_record]['depth_files'][idx]
        color = resources_path / all_data[each_record]['color_files'][idx]
        depth = np.load(str(depth))
        depth = o3d.geometry.Image(depth)
        color = o3d.io.read_image(str(color))
        depth = o3d.t.geometry.Image.from_legacy(depth)
        color = o3d.t.geometry.Image.from_legacy(color)
        rgbd_image = o3d.t.geometry.RGBDImage(color, depth)
        pcd_frame = o3d.t.geometry.PointCloud.create_from_rgbd_image(
                    rgbd_image, 
                    o3c.Tensor(intrinsic, dtype=o3c.Dtype.Float32, device=o3d_device),
                    o3c.Tensor(np.eye(4), dtype=o3c.Dtype.Float32, device=o3d_device),
                    1000, 3,
                    2, False)
        xyz = np.asarray(pcd_frame.to_legacy().points)
        rgb = np.asarray(pcd_frame.to_legacy().colors)
        color_numpy = np.asarray(color.to_legacy())
        print(color_numpy.shape)
        # try:
        label = segment_pcd_from_2d(model, pcd_frame, 
                                    resources_path / all_data[each_record]['color_files'][idx] 
                                    , intrinsic)
        # except Exception as e:
        #    print(resources_path / all_data[each_record]['color_files'][idx])
        #    print(e)
        pcd_with_labels = np.hstack((xyz, rgb, label.reshape(-1, 1)))
        ply_name = resources_path / all_data[each_record]['point_cloud_files'][idx]
        vertex = np.array(
                    [(x, y, z, r, g, b, s) for x, y, z, r, g, b, s in pcd_with_labels],
                    dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
                           ('segment_id', 'i4')])
        ply = PlyData([PlyElement.describe(vertex, 'vertex')], text=True)
        ply.write(str(ply_name))

    # break



In [7]:
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")  # load a pretrained model (recommended for training)

# results = model.train(data="Objects365.yaml", epochs=100, imgsz=640)
image_path =  '/home/capre/disk_4/yutao/data/resources/200a4ee908534e36a0aae0e39e2c8854_color_3.png'
res= model.predict(source=image_path, save=False, verbose=False)
res[0].show()
